In [9]:
def add_1(complement):
    n = len(complement)
    carry = 1
    i = n - 1
    complement = list(complement)

    while i > 0 and carry:
        if complement[i] == '0' and carry == 1:
            complement[i] = '1'
            carry = 0
        if complement[i] == '1' and carry == 1:
            complement[i] = '0'
        i -= 1
        #print(carry)

    return "".join(complement)
    
def decimal_dec_to_bin(x, decimal_digit):
    """
    :param x: is the original number
    :param decimal_digit: the number of digits of decimal part.
    :return:
    """
    x -= int(x)
    bins = ""
    i = 0
    while x and i < decimal_digit:
        x *= 2
        bins += "1" if x >= 1. else "0"
        x -= int(x)
        i += 1
    return bins

def dec_to_bin(x, int_digit, decimal_digit):
    """
    :param x: original number
    :param int_digit: the number of digits of integer part.
    :param decimal_digit: the number of digits of decimal part.
    :return:
    """
    decimal = decimal_dec_to_bin(x, decimal_digit)
    if x >= 0:  # true form = complement
        integer = bin(int(x))[2:]
        add_zero_int = int_digit - len(integer)
        add_zero_integer_complement ="0" + "0" * add_zero_int + integer
    if x < 0:
        integer = bin(int(x))[3:]
        #print("integer: ", integer)
        add_zero_int = int_digit - len(integer)
        #print("add zero:", add_zero_int)
        add_zero_integer_true = "1" + "0" * add_zero_int + integer # true form
        """
        print("add_zero_integer: ", add_zero_integer)
        print("join content: ", [int(r) for r in add_zero_integer[1:]])
        print("join content: ", [~int(r) for r in add_zero_integer[1:]])
        print("join content: ", [0 if int(r) == 1 else 1 for r in add_zero_integer[1:]])
        """

        add_zero_integer_reverse = "1" + "".join(str(0 if int(r) == 1 else 1) for r in add_zero_integer_true[1:]) # radix-minus-one complement
        add_zero_integer_complement = add_1(add_zero_integer_reverse)
        #print(add_zero_integer_complement)

         # radix-minus-one complement
    add_zero_dec = decimal_digit - len(decimal)
    add_zero_dec = decimal + "0" * add_zero_dec

    return add_zero_integer_complement + "." + add_zero_dec

In [26]:
#question, int_digit + decimal_digit == 15 and 1 bit for sign right?


# now, why two zeros before '.'? should be only 1
dec_to_bin(0.5, int_digit=0, decimal_digit=15)

'00.100000000000000'

In [29]:
# set int_digit = 1 and same fraction part as before (just 1 less zero), this time we have a correct answer
dec_to_bin(0.5, int_digit=1, decimal_digit=14)

'00.10000000000000'

In [31]:
# things becomes normal
dec_to_bin(0.5, int_digit=2, decimal_digit=13)

'000.1000000000000'

In [33]:
# now, disaster appears
dec_to_bin(-0.5, int_digit=0, decimal_digit=15)

'1.000000000000000'

In [34]:
# disaster appears again
dec_to_bin(-0.5, int_digit=1, decimal_digit=15)

'10.000000000000000'

In [36]:
# disaster appears again and again :(
dec_to_bin(-0.5, int_digit=2, decimal_digit=15)

'100.000000000000000'